In [2]:
import csv
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [34]:
df = pd.read_csv('IMDB.csv',names = ['id','text','category'])
df.set_index('id', inplace = True)
df.category.value_counts()
df

,text,category
id,,
1,One of the other reviewers has mentioned that ...,positive
2,A wonderful little production br br The filmin...,positive
3,I thought this was a wonderful way to spend ti...,positive
4,Basically theres a family where a little boy J...,negative
5,Petter Matteis Love in the Time of Money is a ...,positive
...,...,...
6545,The Secret of Kells is an independent animated...,positive
6546,This is a cleaver film featuring love through ...,positive
6547,All the boys seem to be sexually aroused by Ma...,negative


In [3]:
df = df[df.category != "NaN"]
df.category.value_counts()

negative    406
positive    392
Name: category, dtype: int64

In [4]:
possible_labels = df.category.unique()
label_dict={}
for index,possible_label in enumerate(possible_labels):
  label_dict[possible_label] = index

In [6]:
df['label'] = df.category.replace(label_dict)
df.head()

,text,category,label
id,,,
1,One of the other reviewers has mentioned that ...,positive,0
2,A wonderful little production br br The filmin...,positive,0
3,I thought this was a wonderful way to spend ti...,positive,0
4,Basically theres a family where a little boy J...,negative,1
5,Petter Matteis Love in the Time of Money is a ...,positive,0


In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(
    df.index.values,
    df.label.values,
    test_size = 0.15,
    random_state = 17,
    
)

In [8]:
df ['data_type'] = ['not_set']*df.shape[0]

In [10]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [11]:
df.groupby(['category', 'label', 'data_type'])

In [12]:
!pip install transformers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

     |████████████████████████████████| 1.4MB 13.6MB/s 
     |████████████████████████████████| 2.9MB 42.8MB/s 
     |████████████████████████████████| 890kB 45.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=065c081c3ee76cbbe8e45f865630950e762d12ec9acc1c25f4688285e53f2536
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [13]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
)

In [14]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type == 'train'].text.values,
    add_special_tokens = True,
    return_attention_mask = True,
    pad_to_max_length =True,
    max_length = 256,
    return_tensors = 'pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type == 'val'].text.values,
    add_special_tokens = True,
    return_attention_mask = True,
    pad_to_max_length =True,
    max_length = 256,
    return_tensors = 'pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type == 'train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type == 'val'].label.values)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.6/dist-packages/transformers/tokenization_utils_base.py:2142: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  FutureWarning,


In [15]:
dataset_train = TensorDataset(input_ids_train,
                              attention_masks_train,labels_train)
dataset_val = TensorDataset(input_ids_val,
                              attention_masks_val,labels_val)

In [16]:
len(dataset_train)
len(dataset_val)

120

In [17]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = len(label_dict),
    output_attentions = False,
    output_hidden_states = False
)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [18]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [19]:
batch_size = 4
dataloader_train = DataLoader(
    dataset_train,
    sampler = RandomSampler(dataset_train),
    batch_size = batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler = RandomSampler(dataset_val),
    batch_size = 32
)

In [20]:
from transformers import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(
    model.parameters(),
    lr = 1e-5,
    eps = 1e-8
)

epochs = 10

schduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps = 0,
    num_training_steps = len(dataloader_train)*epochs
)

In [21]:
import numpy as np

In [22]:
from sklearn.metrics import f1_score

In [23]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [24]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v:k for k,v in label_dict.items()}

    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    for label in np.unique(labels_flat):
      y_preds = preds_flat[labels_flat == label]
      y_true = labels_flat[labels_flat == label]
      print(f'Class:{label_dict_inverse[label]}')
      print(f'Accuracy:{len(y_preds[y_preds == label])}/{len(y_true)}\n')

In [25]:
import random

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)

cpu


In [27]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids':      batch[0],
                  'attention_mask': batch[1],
                  'labels':         batch[2],
                 }

        with torch.no_grad():        
            outputs = model(**inputs)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [1]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(dataloader_train,
                        desc = 'Epoch {:1d}'.format(epoch),
                        leave = False,
                        disable = False)
    for batch in progress_bar:

      model.zero_grad()

      batch = tuple(b.to(device) for b in batch)

      inputs = {
          'input_ids': batch[0],
          'attention_mask': batch[1],
          'labels': batch[2]
      }
      outputs = model(**inputs)

      loss = outputs[0]
      loss_train_total += loss.item()
      loss.backward()

      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
  

      optimizer.step()
      schduler.step()

      progress_bar.set_postfix({'training_loss':'{:.3f}'.format(loss.item()/len(batch))})
    torch.save(model.state_dict(),f'Models/BERT_ft_epoch{epoch}.model')

    tqdm.write('\nEpoch {epoch}')

    loss_train_avg = loss_train_total/len(dataloader)
    tqdm.write(f'Training loss:{loss_train_avg}')

    val_loss.predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions.true_vals)
    tqdm.write(f'Validation loss:{val_loss}')
    tqdm.write(f'F1 score (weighted):{val_f1}')


NameError: ignored

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                      num_labels=len(label_dict),
                                                      output_attentions=False,
                                                      output_hidden_states=False)

In [ ]:
model.to(device)
pass

In [ ]:
model.load_state_dict(
    torch.load('finetuned_bert_epoch_1_gpu_trained.model',
               map_location = torch.device('cpu'))
)

In [ ]:
_,predictions, true_vals = evaluate(dataloader_val)
accuracy_per_class(predictions, true_vals)